In [1]:
import platform
import pandas as pd
import json, csv
import pickle
from datetime import datetime
from operator import itemgetter
from pprint import pprint
import importlib
import itertools
from copy import deepcopy
import numpy as np
import math, time, collections, os, errno, sys, code, random
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import mixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import shutil
import configparser
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
def calculate_degree_normalized_matrix(matrix):

    ### calculate degree matrix

    D = np.zeros(matrix.shape)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[0]):
            D[i][i] += matrix[i][j]

    #print(D)


    ### check whether D is a singular matrix

    if np.linalg.cond(D) < 1 / sys.float_info.epsilon:
        D = np.sqrt(np.linalg.inv(D))
        return np.matmul(np.matmul(D, matrix), D)
    else:
        #print("so sad")
        return matrix

In [3]:
def calculate_E_and_M(G):
        
        c = 0.5
        N = 7
        t_w = 1
        
        ### calculate E
        
        E = (1 - c) * np.linalg.inv(np.identity(N * t_w) - c * G)
        #print("E: \n" + str(E))
        
        
        ### calculate M
        
        M = []
        for i in range(len(G)):
            M.append([])
            for j in range(len(G[i])):
                if G[i][j] != 0:
                    M[i].append(1)
                else:
                    M[i].append(G[i][j])
        M = np.asarray(M)
        #print("M: \n" + str(M))
        
        return E, M

In [4]:
G = np.asarray([[0, 1, 0, 0, 1, 0, 0], \
                [1, 0, 1, 1, 0, 0, 0], \
                [0, 1, 0, 0, 0, 0, 0], \
                [0, 1, 0, 0, 0, 1, 0], \
                [1, 0, 0, 0, 0, 1, 0], \
                [0, 0, 0, 1, 1, 0, 1], \
                [0, 0, 0, 0, 0, 1, 0]])
G = calculate_degree_normalized_matrix(G)
print(G)

[[0.         0.40824829 0.         0.         0.5        0.
  0.        ]
 [0.40824829 0.         0.57735027 0.40824829 0.         0.
  0.        ]
 [0.         0.57735027 0.         0.         0.         0.
  0.        ]
 [0.         0.40824829 0.         0.         0.         0.40824829
  0.        ]
 [0.5        0.         0.         0.         0.         0.40824829
  0.        ]
 [0.         0.         0.         0.40824829 0.40824829 0.
  0.57735027]
 [0.         0.         0.         0.         0.         0.57735027
  0.        ]]


In [5]:
E, M = calculate_E_and_M(G)
print(E)
print(M)

[[0.56469003 0.13369856 0.03859545 0.03571429 0.14959569 0.04126499
  0.01191218]
 [0.13369856 0.60444744 0.17448895 0.13122266 0.04126499 0.0384097
  0.01108793]
 [0.03859545 0.17448895 0.55037062 0.03788072 0.01191218 0.01108793
  0.00320081]
 [0.03571429 0.13122266 0.03788072 0.55357143 0.03571429 0.13122266
  0.03788072]
 [0.14959569 0.04126499 0.01191218 0.03571429 0.56469003 0.13369856
  0.03859545]
 [0.04126499 0.0384097  0.01108793 0.13122266 0.13369856 0.60444744
  0.17448895]
 [0.01191218 0.01108793 0.00320081 0.03788072 0.03859545 0.17448895
  0.55037062]]
[[0. 1. 0. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [6]:
np.set_printoptions(precision = 4, suppress = True)

In [7]:
A = np.asarray([[0, 1, 0, 0, 1, 0, 0], \
                [1, 0, 1, 1, 0, 0, 0], \
                [0, 1, 0, 0, 0, 0, 0], \
                [0, 1, 0, 0, 0, 1, 0], \
                [1, 0, 0, 0, 0, 1, 0], \
                [0, 0, 0, 1, 1, 0, 1], \
                [0, 0, 0, 0, 0, 1, 0]])
b = np.asarray([1, 0, 0, 0, 0, 0, 0])
s = np.matmul(np.linalg.inv(E), b)
print(s)

[ 2.     -0.4082 -0.     -0.     -0.5     0.     -0.    ]


In [8]:
A = np.asarray([[0, 0, 0, 0, 1, 0, 0], \
                [0, 0, 1, 1, 0, 0, 0], \
                [0, 1, 0, 0, 0, 0, 0], \
                [0, 1, 0, 0, 0, 1, 0], \
                [1, 0, 0, 0, 0, 1, 0], \
                [0, 0, 0, 1, 1, 0, 1], \
                [0, 0, 0, 0, 0, 1, 0]])
B = G - A
b = np.asarray([1, 1, 0, 0, 0, 0, 0])
s = np.matmul(np.linalg.inv(E), b)
print(s)

[ 1.5918  1.5918 -0.5774 -0.4082 -0.5    -0.     -0.    ]


In [9]:
A = np.asarray([[0, 0, 0, 0, 0, 0, 0], \
                [0, 0, 1, 1, 0, 0, 0], \
                [0, 1, 0, 0, 0, 0, 0], \
                [0, 1, 0, 0, 0, 1, 0], \
                [0, 0, 0, 0, 0, 1, 0], \
                [0, 0, 0, 1, 1, 0, 1], \
                [0, 0, 0, 0, 0, 1, 0]])
B = G - A
b = np.asarray([1, 1, 0, 0, 1, 0, 1])
s = np.matmul(np.linalg.inv(E), b)
print(s)

[ 1.0918  1.5918 -0.5774 -0.4082  1.5    -0.9856  2.    ]


In [10]:
A = np.asarray([[0, 0, 0, 0, 0, 0, 0], \
                [0, 0, 0, 1, 0, 0, 0], \
                [0, 0, 0, 0, 0, 0, 0], \
                [0, 1, 0, 0, 0, 1, 0], \
                [0, 0, 0, 0, 0, 1, 0], \
                [0, 0, 0, 1, 1, 0, 1], \
                [0, 0, 0, 0, 0, 1, 0]])
B = G - A
b = np.asarray([1, 1, 1, 0, 1, 0, 0])
s = np.matmul(np.linalg.inv(E), b)
print(s)

[ 1.0918  1.0144  1.4226 -0.4082  1.5    -0.4082  0.    ]
